# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import requests
from IPython.display import Markdown

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:8]}")
else:
    print("Groq API Key not set")

openrouter_url = "https://openrouter.ai/api/v1"

client = OpenAI(api_key=openai_api_key, base_url=openrouter_url)
# Groq client for Whisper
groq_client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

GPT_MODEL = "openai/gpt-5-nano"
GEMINI_MODEL = "google/gemini-2.5-pro-preview"

In [ ]:
system_message = f"""
You are an assistant who help writing CV. Consider including information about the user's professional 
experience, education, skills, optional certifications and interests. 
Include only the information that is relevant to the position the candidate is applying for.
Try to make your CV not too long and contain the most important information for the employer.
"""

In [ ]:
def transcribe_audio(audio_path, language):
    if audio_path is None:
        return ""

    print(f"Transcribing audio: {audio_path} in language: {language}")

    try:
        with open(audio_path, 'rb') as audio_file:
            transcription = groq_client.audio.transcriptions.create(
                model="whisper-large-v3-turbo",
                file=audio_file,
                language=language
            )
        print(f"Transcription result: {transcription.text}")
        return transcription.text
    except Exception as e:
        print(f"Transcription error: {e}")
        return f"⚠️ Transcription error: str{e}"

In [ ]:
def chat(message, history, selected_model):
    messages = [{"role": "system", "content": system_message}] + history

    if selected_model == "GPT-5-nano":
        model = GPT_MODEL
    elif selected_model == "gemini-2.5-pro-preview":
        model = GEMINI_MODEL
    else:
        yield history + [{"role": "assistant", "content": "⚠️ Unknown model selected."}]
        return

    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        try:
            content = chunk.choices[0].delta.content or ""
            response += content
            updated_history = history + [{"role": "assistant", "content": response}]
            yield updated_history
        except Exception as e:
            print(f"Streaming error: {e}")

In [ ]:
def do_entry(message, audio, language, history):
    if audio is not None:
        transcribed_text = transcribe_audio(audio, language)
        if transcribed_text and not transcribed_text.startswith("⚠️"):
            message = transcribed_text
        else:
            # Show transcribtion error
            history = history + [[f"🎤 Audio", transcribed_text]]
            return "", None, history

    if not message or message.strip() == "":
        return "", None, history
    
    history = history + [[message, None]]
    return "", None, history

#Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as UI:
    gr.Markdown("# CV-writing AI assistant")
    gr.Markdown("**Write or speak your CV information!**")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=500, type="tuples", label="CV Assistant Chat", value=[])

            with gr.Row():
                audio_input = gr.Audio(
                    sources=["microphone"],
                    type="filepath",
                    label="🎤 Click to record (speak after clicking)",
                    show_label=True,
                    interactive=True
                )

            with gr.Row():
                entry = gr.Textbox(
                    label="Or type your message here",
                    placeholder="I am a software developer with 5 years of experience..",
                    lines=3
                )

            with gr.Row():
                submit_button = gr.Button("✉ Send", variant="primary", scale=2)
                clear_button = gr.Button("Clear", scale=1)

        with gr.Column(scale=1):
            model_dropdown = gr.Dropdown(
                choices=["GPT-5-nano", "gemini-2.5-pro-preview"],
                label="🤖 Choose AI Model"
            )
            language_dropdown = gr.Dropdown(
                choices=[
                    ("🇵🇱 Polski", "pl"),      
                    ("🇬🇧 English", "en"),
                    ("🇩🇪 Deutsch", "de"),
                ],
                value="en",
                label="Audio Language"
            )

        gr.Markdown("""
            ### How to use:
            
            **Option 1: Voice** 🎤
            1. Click microphone icon
            2. Speak your CV info
            3. Click Send
            
            **Option 2: Text** ⌨️
            1. Type your message
            2. Click Send or press Enter
        """)

    def handle_submit(message, audio, language, history, model):
        if history is None:
            history = []
        if not isinstance(history, list):
            print(f"WARNING: Converting history from {type(history)} to list")
            history = []
            
        new_entry, new_audio, updated_history = do_entry(message, audio, language, history)
        yield new_entry, new_audio, updated_history

        if len(updated_history) > len(history) and updated_history:
            last_pair = updated_history[-1]  

            if isinstance(last_pair, list) and len(last_pair) >= 1:
                last_user_message = last_pair[0]
                history_without_last = updated_history[:-1] 
                
                for response_history in chat(last_user_message, history_without_last, model):
                    yield new_entry, new_audio, response_history

    submit_button.click(
        handle_submit,
        inputs=[entry, audio_input, chatbot, model_dropdown],
        outputs=[entry, audio_input, chatbot]
    )
    entry.submit(
        handle_submit,
        inputs=[entry, audio_input, language_dropdown, chatbot, model_dropdown],
        outputs=[entry, audio_input, chatbot]
    )
    clear_button.click(
        lambda: ("", None, []),
        inputs=None,
        outputs=[entry, audio_input, chatbot],
        queue=False
    )

UI.launch()